In [ ]:
# 安裝 selenium  webdriver_manager 套件
!pip install selenium
!pip install webdriver_manager

In [1]:
# 導入套件
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementNotInteractableException 

In [2]:
# 設定學號、搶課密碼
student_id = 'XXXXXX'
password = 'XXXXXX'

# 設定搶課順序(1~10) ex: seq = [1, 2, 3, 4, 5]
seq = [2, 3, 5, 1, 4]

In [3]:
# 創建瀏覽器
def create_driver(driver_path="chromedriver.exe"):
    try:
        # 自動安裝 chromedriver
        return webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    except Exception:
        # 自動安裝失敗，使用本地安裝的 chromedriver
        return webdriver.Chrome(service=Service(driver_path))

# 登入
def login(student_id, password):
    # 設定等候時間
    wait = WebDriverWait(driver, 1)

    # 輸入學號、選課密碼，點擊登入
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="ContentPlaceHolder1_ed_StudNo"]'))).send_keys(student_id)
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="ContentPlaceHolder1_ed_pass"]'))).send_keys(password)
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="ContentPlaceHolder1_BtnLoginNew"]'))).click()

    # 接受警示框(冒用情形警示框，確認)
    wait.until(EC.alert_is_present()).accept()

    try:
        # 取消填寫教學意見，接受警示框(冒用情形警示框，確認)
        wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div[3]/div/button[2]'))).click()
        wait.until(EC.alert_is_present()).accept()
    except:
        pass

#　創建瀏覽器
driver = create_driver()

# 打開指定網站(網路選課系統)
driver.get("https://sys.ndhu.edu.tw/AA/CLASS/subjselect/")

# 登入
login(student_id, password)


In [ ]:
# 自動選課(可以提早運行)
def auto_select_course(driver, seq):
    # 點擊選課
    driver.find_elements(By.XPATH, '//*[@id="ContentPlaceHolder1_HyperLink1"]')[0].click()

    # 點擊查詢預排課程
    driver.find_elements(By.XPATH, '//*[@id="ContentPlaceHolder1_Button7"]')[0].click()
    time.sleep(0.1)

    # 獲取所有加選元素
    elements = driver.find_elements(By.XPATH, '//*[@id="ContentPlaceHolder1_grd_subjs"]/tbody/tr/td[1]/input')
    first_element = elements[seq[0] - 1]

    # 連續點擊第一個課程
    while True:
        try:
            # 點擊加選，接受警示框(確認)
            elements[seq[0] - 1].click()
            driver.switch_to.alert.accept()
        except ElementNotInteractableException:
            # 按鈕消失時，跳出迴圈
            break

    # 點擊剩餘課程
    for i in seq[1:]:
        # 點擊加選，接受警示框(確認)
        elements[i - 1].click()
        driver.switch_to.alert.accept()

auto_select_course(driver, seq)